# Import Datasets

## Extract Relevant Columns

In [401]:
import pandas as pd
import numpy as np

In [402]:
#this data was constructed without embedded data, and is easier to identify by column name
raw_data = pd.read_csv("data/april_3_raw.csv") 
raw_data = raw_data.loc[1:,] #drop header

In [403]:
#this data was constructed by tools > embedded data > select the embedded values
meta_data = pd.read_csv("data/april_3_meta.csv")
meta_data = raw_data.loc[2:,] #drop header
meta_data = meta_data[["_recordId","social_media_group", "social_media_year"]]
meta_data = meta_data.rename(columns = {"_recordId": "id"})
meta_data = meta_data.set_index("id")

In [404]:
#cleaning up raw data

#year, gender, gender -other, race, primary role, year at berk, 
#transfer, degree, access SM - read access SM - write
demo = ["_recordId", "QID214_TEXT", "QID367", "QID367_6_TEXT", "QID16", "QID766", "QID767",
           "QID768", "QID341", "QID251_1", "QID251_2"]

#support mental health, social distancing, drag to 50, demeaning womxn
#health eating habits, stop a friend SVSH, recall, receive more info, contacted
outcomes = ["QID425_1",	"QID426_1",	"QID427_1",	"QID428_1",
            "QID429_1",	"QID430_1", "QID432", "QID434", "QID769_TEXT"]

rename_cols = {"_recordId": "id", "QID214_TEXT": "year", "QID367": "gender", "QID367_6_TEXT": "gender-text", 
               "QID16": "race", "QID766": "role", "QID767": "student_year", "QID768": "transfer", 
               "QID341":"highest_degree", "QID251_1":"read_sm", "QID251_2":"write_sm", 
               "QID425_1": "mental_health",	"QID426_1": "social_distancing", "QID427_1":"drag_50",	
               "QID428_1":"sexist_comment", "QID429_1":"support_eating", "QID430_1":"stop_svsh", 
               "QID434": "treat_recall", "QID432": "control_recall", "QID769_TEXT":"receive_info"}
columns = demo + outcomes
raw_data = raw_data[columns]
raw_data = raw_data.rename(columns = rename_cols).set_index("id")

In [405]:
#join metadata onto raw_data
df = raw_data.join(meta_data)

In [406]:
df.head()

,year,gender,gender-text,race,role,student_year,transfer,highest_degree,read_sm,write_sm,...,social_distancing,drag_50,sexist_comment,support_eating,stop_svsh,control_recall,treat_recall,receive_info,social_media_group,social_media_year
id,,,,,,,,,,,,,,,,,,,,,
R_3kpkeubEVJMXVeY,NaN,NaN,NaN,Native American,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R_2rORoun447CrzQB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13,50,70,NaN,92,NaN,"bear,treatment,clouds",NaN,not_cis_man_treatment,NaN
R_2QSquguoMN5Dj7A,1986,Cisgender Man,NaN,Asian,NaN,NaN,NaN,"Advanced degree (Master's, Doctorate)",Weekly,Weekly,...,68,66,65,63,58,"bear,wellness,Click to write Choice 5",NaN,NaN,cis_man_control,NaN
R_u1yNIpPNbXY8GXv,1985,Cisgender Man,NaN,Non-Hispanic White,NaN,NaN,NaN,Some college,More than once a day,Daily,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
R_2Yn2e6TZ2jGGDgh,1991,Cisgender Man,NaN,Non-Hispanic White,NaN,NaN,NaN,Bachelor's degree,More than once a day,Less than Weekly,...,77,50,79,81,91,NaN,"bear,treatment,clouds",NaN,cis_man_treatment,NaN


## Apply Exclusion Criteria

In [407]:
df = df[df["drag_50"] == '50'] #this indicator tells us participants did not read instructions
df = df[df['student_year'].notna()] #we only care about students at berk
df = df[df['transfer'].notna()]

## Binarize transfer column

In [408]:
transfer = df["transfer"].str.replace("No", "0") #codes "not transfer" as baseline
transfer = transfer.str.replace("Yes", "1") 
df["transfer"] = transfer #replace the column with cleaned column

## Clean up Gender Columns

In [409]:
df["gender-text"].value_counts() #check to see if anyone wrote in their gender. if so, may have to hand-code into or add into pref not to disclose

Series([], Name: gender-text, dtype: int64)

In [410]:
gender = df['gender'].fillna(df['gender-text']) #bring in any write in genders into the gender column 

In [411]:
#check to ensure none of the gender-text values denote that the individual is a cis man because the following code
#automatically codes anyone who wrote in their gender as NOT a cis man

In [412]:
gender = gender.str.replace("Cisgender Man", "0") #code cisman as baseline
gender = gender.str.replace("[\D].*", "1") #code all other genders as 1
gender #may have to figure out how to handle nulls

id
R_31cuvH38jf2jrSO    0
R_2fiX6Cr8V61UoyO    0
R_3DnYKTNotWcDO2J    1
R_2BfIDrbZGrMAQDL    0
R_1F3Yr6S1FAsxpVp    1
R_1MXLFRa4rkKt20c    0
Name: gender, dtype: object

In [413]:
df["gender"] = gender #replace the column with cleaned column
df = df.drop("gender-text", axis = 1) #don't need this anymore

## One-Hot Encode Categorical Variables

In [414]:
def OHE(df, column, prefix = None):
    X = pd.get_dummies(df[column].str.lower(), prefix = prefix) #currently, results in multicollinear columns.
    df = df.join(X, on = "id")
    df = df.drop(column, axis = 1) #don't need this anymore
    return df

In [415]:
df = OHE(df, "race")
df = OHE(df, "role")
df = OHE(df, "student_year")
df = OHE(df, "highest_degree")
df = OHE(df, "read_sm", prefix = "read")
df = OHE(df, "write_sm", prefix = "write")
df.head()

,year,gender,transfer,mental_health,social_distancing,drag_50,sexist_comment,support_eating,stop_svsh,control_recall,...,undergraduate student,1st year,2nd year,3rd year,some college,read_daily,read_more than once a day,read_weekly,write_daily,write_less than weekly
id,,,,,,,,,,,,,,,,,,,,,
R_31cuvH38jf2jrSO,1978,0,0,95,50,50,75,80,60,"bear,control,clouds",...,1,0,1,0,1,1,0,0,0,1
R_2fiX6Cr8V61UoyO,2000,0,1,80,10,50,25,60,50,"bear,control,clouds",...,1,0,0,1,1,0,1,0,0,1
R_3DnYKTNotWcDO2J,1996,1,1,72,15,50,40,30,91,NaN,...,1,1,0,0,1,0,1,0,0,1
R_2BfIDrbZGrMAQDL,1996,0,1,88,20,50,90,71,62,bear,...,1,1,0,0,1,0,1,0,0,1
R_1F3Yr6S1FAsxpVp,2003,1,1,78,81,50,74,83,78,NaN,...,1,0,1,0,1,1,0,0,0,1


In [416]:
rename = {'black or african american': 'black', 'hispanic or latino': 'latino', 'non-hispanic white': 'white',
       'undergraduate student': 'undergrad_student', 'graduate student': 'grad_student','1st year':"1st_year", 
          '2nd year': "2nd_year", '3rd year': "3rd_year", 'some college': "some_college", 'read_daily': "read_daily", 'read_more than once a day': 'read_daily_more',
       'write_less than weekly':"write_less_weekly"}

df = df.rename(columns = rename)

## Operationalize Outcomes

In [417]:
df["outcome"] = abs(91 - df["stop_svsh"].astype(int))
df.head()

,year,gender,transfer,mental_health,social_distancing,drag_50,sexist_comment,support_eating,stop_svsh,control_recall,...,1st_year,2nd_year,3rd_year,some_college,read_daily,read_daily_more,read_weekly,write_daily,write_less_weekly,outcome
id,,,,,,,,,,,,,,,,,,,,,
R_31cuvH38jf2jrSO,1978,0,0,95,50,50,75,80,60,"bear,control,clouds",...,0,1,0,1,1,0,0,0,1,31
R_2fiX6Cr8V61UoyO,2000,0,1,80,10,50,25,60,50,"bear,control,clouds",...,0,0,1,1,0,1,0,0,1,41
R_3DnYKTNotWcDO2J,1996,1,1,72,15,50,40,30,91,NaN,...,1,0,0,1,0,1,0,0,1,0
R_2BfIDrbZGrMAQDL,1996,0,1,88,20,50,90,71,62,bear,...,1,0,0,1,0,1,0,0,1,29
R_1F3Yr6S1FAsxpVp,2003,1,1,78,81,50,74,83,78,NaN,...,0,1,0,1,1,0,0,0,1,13


In [419]:
df["receive_info"] = df["receive_info"].str.replace(".*", "1")
df["receive_info"] = df["receive_info"].fillna("0")
df["receive_info"] = df["receive_info"].astype(int)

## Binarize Treatment Column

In [420]:
treatment = df["social_media_group"]
treatment = treatment.str.replace(".*(control)", "0")
treatment = treatment.str.replace(".*(treatment)", "1")

df["treatment"] = treatment.astype(int)
df = df.drop("social_media_group", axis = 1) #don't need this anymore


## Fix Recall

In [421]:
df.columns

Index(['year', 'gender', 'transfer', 'mental_health', 'social_distancing',
       'drag_50', 'sexist_comment', 'support_eating', 'stop_svsh',
       'control_recall', 'treat_recall', 'receive_info', 'social_media_year',
       'asian', 'black', 'latino', 'white', 'undergrad_student', '1st_year',
       '2nd_year', '3rd_year', 'some_college', 'read_daily', 'read_daily_more',
       'read_weekly', 'write_daily', 'write_less_weekly', 'outcome',
       'treatment'],
      dtype='object')

In [422]:
#TDL for Data Cleaning

#1. one hot encode all categorical columns DONE
#2. operationalize the recall column - "did they recall treatment or control" 1, 0 DONE
#2a. *maybe* think about adding other recall attributes DONE
#3. what to fill in for null values on the block randomized questions 
# (drop all rows where this is the case)
#4. Drop ppl in drag_50 column where answer != 50 - DONE 
#5. Change "social_media_group" to 1 or 0 for treatment vs control DONE
#6. Compute outcome "Stop_SVSH" absolute difference column DONE

In [423]:
recall = df['treat_recall'].fillna(df['control_recall']) #bring in any write in genders into the gender column 

recall = recall.str.replace(".*(control|treatment).*", "1")
recall = recall.str.replace("[\D].*", "0") 

df["recall"] = recall

df = df.drop(columns = ["treat_recall", 'control_recall']) #don't need this anymore


In [424]:
df.columns

Index(['year', 'gender', 'transfer', 'mental_health', 'social_distancing',
       'drag_50', 'sexist_comment', 'support_eating', 'stop_svsh',
       'receive_info', 'social_media_year', 'asian', 'black', 'latino',
       'white', 'undergrad_student', '1st_year', '2nd_year', '3rd_year',
       'some_college', 'read_daily', 'read_daily_more', 'read_weekly',
       'write_daily', 'write_less_weekly', 'outcome', 'treatment', 'recall'],
      dtype='object')

In [425]:
df.head()

,year,gender,transfer,mental_health,social_distancing,drag_50,sexist_comment,support_eating,stop_svsh,receive_info,...,3rd_year,some_college,read_daily,read_daily_more,read_weekly,write_daily,write_less_weekly,outcome,treatment,recall
id,,,,,,,,,,,,,,,,,,,,,
R_31cuvH38jf2jrSO,1978,0,0,95,50,50,75,80,60,0,...,0,1,1,0,0,0,1,31,0,1
R_2fiX6Cr8V61UoyO,2000,0,1,80,10,50,25,60,50,0,...,1,1,0,1,0,0,1,41,0,1
R_3DnYKTNotWcDO2J,1996,1,1,72,15,50,40,30,91,0,...,0,1,0,1,0,0,1,0,1,1
R_2BfIDrbZGrMAQDL,1996,0,1,88,20,50,90,71,62,0,...,0,1,0,1,0,0,1,29,0,0
R_1F3Yr6S1FAsxpVp,2003,1,1,78,81,50,74,83,78,0,...,0,1,1,0,0,0,1,13,1,1


In [427]:
df.to_csv("data/clean_data.csv")